In [18]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Linear, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.neighbors import kneighbors_graph
import torch.nn.functional as F
from torch.optim import Adam

In [19]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 
           'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
data = pd.read_csv(url, header=None, names=columns)

In [20]:
columns_to_check = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in columns_to_check:
    data[col] = data[col].replace(0, np.nan)
    data[col].fillna(data[col].mean(), inplace=True)

scaler = StandardScaler()
X = scaler.fit_transform(data.drop(columns=['Outcome']))
y = data['Outcome'].values



In [21]:

X_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(y, dtype=torch.long)
k = 10
adjacency_matrix = kneighbors_graph(X, n_neighbors=k, mode='connectivity', include_self=True)
edge_index = torch.tensor(np.array(adjacency_matrix.nonzero()), dtype=torch.long)

graph_data = Data(x=X_tensor, edge_index=edge_index, y=y_tensor)


In [22]:
train_idx, test_idx = train_test_split(range(len(graph_data.y)), test_size=0.2, random_state=42, stratify=graph_data.y)
train_mask = torch.tensor(train_idx, dtype=torch.long)
test_mask = torch.tensor(test_idx, dtype=torch.long)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [23]:
class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)  # لایه سوم
        self.fc = Linear(hidden_dim, output_dim)
        self.dropout = Dropout(0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
graph_data = graph_data.to(device)

input_dim = X.shape[1]
hidden_dim = 64
output_dim = len(np.unique(y))

model = GNNModel(input_dim, hidden_dim, output_dim).to(device)
optimizer = Adam(model.parameters(), lr=0.005, weight_decay=5e-5)
criterion = torch.nn.NLLLoss(weight=class_weights.to(device))

In [26]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(graph_data)
    loss = criterion(out[train_mask], graph_data.y[train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()
def test():
    model.eval()
    out = model(graph_data)
    pred = out.argmax(dim=1)
    correct = pred[test_mask].eq(graph_data.y[test_mask]).sum().item()
    acc = correct / test_mask.size(0)
    return acc


In [27]:
for epoch in range(300):  # افزایش تعداد epochs
    loss = train()
    acc = test()
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Accuracy: {acc:.4f}")


Epoch 1, Loss: 0.7129, Accuracy: 0.6104
Epoch 2, Loss: 0.6593, Accuracy: 0.5779
Epoch 3, Loss: 0.6348, Accuracy: 0.5974
Epoch 4, Loss: 0.6161, Accuracy: 0.6299
Epoch 5, Loss: 0.5786, Accuracy: 0.6818
Epoch 6, Loss: 0.5529, Accuracy: 0.6818
Epoch 7, Loss: 0.5575, Accuracy: 0.6948
Epoch 8, Loss: 0.5272, Accuracy: 0.6883
Epoch 9, Loss: 0.5432, Accuracy: 0.6818
Epoch 10, Loss: 0.5329, Accuracy: 0.6753
Epoch 11, Loss: 0.5409, Accuracy: 0.6753
Epoch 12, Loss: 0.5268, Accuracy: 0.6948
Epoch 13, Loss: 0.5360, Accuracy: 0.7013
Epoch 14, Loss: 0.5157, Accuracy: 0.6948
Epoch 15, Loss: 0.5090, Accuracy: 0.7078
Epoch 16, Loss: 0.5111, Accuracy: 0.7013
Epoch 17, Loss: 0.5093, Accuracy: 0.7078
Epoch 18, Loss: 0.5087, Accuracy: 0.7143
Epoch 19, Loss: 0.5104, Accuracy: 0.7403
Epoch 20, Loss: 0.4979, Accuracy: 0.7208
Epoch 21, Loss: 0.4963, Accuracy: 0.7143
Epoch 22, Loss: 0.4959, Accuracy: 0.7143
Epoch 23, Loss: 0.4837, Accuracy: 0.7273
Epoch 24, Loss: 0.4884, Accuracy: 0.7013
Epoch 25, Loss: 0.4832, A